In [434]:
'''
1. Create a table showing constituent (stocks) risk analysis in the equal-weight portfolio analysis as of the
current date.
a. Column 1 – Ticker [Complete]
b. Column 2 – Portfolio Weight (equally weighted) [Complete]
c. Column 3 – Annualized Volatility (using trailing 3-months) [Complete]
d. Column 4 – Beta against SPY (using trailing 12-months) [Complete]
e. Column 5 – Beta against IWM (using trailing 12-months) [Complete]
f. Column 6 – Beta against DIA (using trailing 12-months [Complete]
g. Column 7 – Average Weekly Drawdown (52-week Low minus 52-week High) / 52-week High [Complete]
h. Column 8 – Maximum Weekly Drawdown (52-week Low minus 52-week High) / 52-week High [Complete]
i. Column 9 – Total Return (using trailing 10-years) [Complete]
j. Column 10 – Annualized Total Return (using trailing 10-years) [Complete]

2. Create a table showing Portfolio Risk against the three ETFs:
a. Column 1 – ETF Ticker [Complete]
b. Column 2 – Correlation against ETF [Complete]
c. Column 3 – Covariance of Portfolio against ETF  [Complete]
d. Column 4 – Tracking Errors (using trailing 10-years) [Complete]
e. Column 5 – Sharpe Ratio (using current risk-free rate) [ Complete]
f. Column 6 – Annualized Volatility (252 days) Spread (Portfolio Volatility – ETF Volatility)

3. Create a correlation matrix showing the correlations between the equal-weighted portfolio created from
your 7 assets, 3 ETFs, and your 7 individual stocks.

'''


import pandas as pd
import yfinance as yf
import numpy as np

In [435]:
# Tickers on the NYSE traded assets, 7 stocks and 3 ETFS
tickers = ['NVDA', 'TSLA', 'GME','AMD','MSFT','META','WMT']
etfs = ['SPY', 'IWM', 'DIA']

In [436]:
# Downloading the historical data, 10 years is a safe number
start_date = '2012-11-23'
end_date = '2022-11-23'

data = yf.download(tickers + etfs, start = start_date, end = end_date)['Adj Close']



[*********************100%***********************]  10 of 10 completed


In [437]:
# Portfolio Weight: A percentage of an investment portfolio that a specific asset or holding represents
# Formula = [(Stock's Value) / (Total Portfolio Value)]. In this case they all weigh the same amount.
'''
portfolioWeight = np.ones(len(tickers)) / len(tickers)

# Create DataFrame with tickers as index and portfolio weights as a column
df = pd.DataFrame({'Weight': portfolioWeight}, index=tickers)
print(df)
'''
portfolio = pd.DataFrame(index=tickers)
n = len(tickers)
portfolioWeight = (n/100) 
portfolio['Portfolio Weight (%)'] = portfolioWeight
portfolio


,Portfolio Weight (%)
NVDA,0.07
TSLA,0.07
GME,0.07
AMD,0.07
MSFT,0.07
META,0.07
WMT,0.07


Since there are 7 assets and there isn't a specific money distribution, they all weigh the same in the portfolio as shown in the dataframe.

In [438]:
''' 
Credit: https://blog.quantinsti.com/volatility-and-measures-of-risk-adjusted-return-based-on-volatility/
Annualized Volatilty is used to indicate how much the value of an investment is likely to fluctuate
Formula = Standard Deviation x sqrt(252) where 252 is the annual that excludes weekends and holidays
.rolling is a function built into pandas that allows us to apply operations over a window of a specified size

'''

portfolio['Annualized Volatility'] = data.pct_change()[-63:].std() * np.sqrt(252)
portfolio




,Portfolio Weight (%),Annualized Volatility
NVDA,0.07,0.634418
TSLA,0.07,0.557269
GME,0.07,0.746471
AMD,0.07,0.642289
MSFT,0.07,0.400705
META,0.07,0.716671
WMT,0.07,0.246433


Most recent annualized volatility of the 10 Tickers in a 3-month trail (91 window)

In [439]:
# Beta = Covariance (Stock Returns, Market Returns) / Variance (Market Returns)

returns = data.pct_change() 

# Credit: https://github.com/CCNY-Analytics-and-Quant/PortfolioAnalysis-Rifat_Kaljang/blob/main/PortfolioAnalysis.ipynb
# This extracts the covariance between the asset and the specified ETF from the covariance matrix.
for etf in etfs:
    portfolio['Beta Against ' + etf] = returns[-252:].cov()[etf] / returns[-252:][etf].var()

portfolio




,Portfolio Weight (%),Annualized Volatility,Beta Against SPY,Beta Against IWM,Beta Against DIA
NVDA,0.07,0.634418,2.214848,1.839605,2.376997
TSLA,0.07,0.557269,1.780724,1.539481,1.788735
GME,0.07,0.746471,1.957305,1.972465,2.045533
AMD,0.07,0.642289,2.067816,1.721387,2.218046
MSFT,0.07,0.400705,1.259122,0.916869,1.393819
META,0.07,0.716671,1.702263,1.337453,1.773996
WMT,0.07,0.246433,0.428176,0.265739,0.570775


In [440]:
weekly_returns = data.pct_change(5).mean()
weekly_returns

Ticker
AMD     0.010533
DIA     0.002593
GME     0.016869
IWM     0.002353
META    0.004266
MSFT    0.005297
NVDA    0.009819
SPY     0.002684
TSLA    0.012102
WMT     0.002325
dtype: float64

In [441]:
weekly_highs = data.pct_change(5).max()
weekly_highs

Ticker
AMD     0.477778
DIA     0.198625
GME     7.883180
IWM     0.156004
META    0.440107
MSFT    0.178336
NVDA    0.363288
SPY     0.173582
TSLA    0.564756
WMT     0.153085
dtype: float64

In [442]:
weekly_lows = data.pct_change(5).min()
weekly_lows

Ticker
AMD    -0.326808
DIA    -0.188748
GME    -0.803785
IWM    -0.239954
META   -0.309781
MSFT   -0.163649
NVDA   -0.283561
SPY    -0.179693
TSLA   -0.430459
WMT    -0.194867
dtype: float64

In [443]:
'''
Maximum Drawdown (MDD) = (Trough Value - Peak Value) / (Peak Value)
'''
drawdowns = (data - weekly_highs)/weekly_highs
drawdowns

Ticker,AMD,DIA,GME,IWM,META,MSFT,NVDA,SPY,TSLA,WMT
Date,,,,,,,,,,
2012-11-23 00:00:00+00:00,3.081396,508.053081,-0.405692,436.304063,53.368475,125.261146,-0.244092,654.321340,2.792790,117.759615
2012-11-26 00:00:00+00:00,2.913954,506.603139,-0.405692,437.717030,57.763262,123.848085,-0.230752,652.930059,2.809316,117.269040
2012-11-27 00:00:00+00:00,2.934884,503.310770,-0.408794,437.173498,58.238987,122.435078,-0.227577,649.592285,2.795150,116.575426
2012-11-28 00:00:00+00:00,3.102326,507.073175,-0.414112,440.868918,58.714711,123.711378,-0.221224,654.831060,2.922639,118.368680
2012-11-29 00:00:00+00:00,3.269768,509.032755,-0.419209,445.922824,60.889447,121.842527,-0.235199,657.891009,2.976940,118.825416
...,...,...,...,...,...,...,...,...,...,...
2022-11-16 00:00:00+00:00,151.162791,1629.331393,2.442773,1147.306538,255.505948,1335.262428,42.754492,2211.946464,329.974925,313.231622
2022-11-17 00:00:00+00:00,153.674429,1629.476894,2.521421,1136.649130,251.473598,1334.986062,42.113715,2205.175275,323.334887,312.173621
2022-11-18 00:00:00+00:00,152.983728,1638.960592,2.501125,1144.003387,252.832825,1332.443323,41.376678,2215.192113,318.058277,316.892412


In [444]:
# Window = 5 for weekly, .mean to calculate weekly drawdown average, .iloc[-1] to pull most recent
'''
average_weekly_drawdown = drawdowns.rolling(window=5).mean().iloc[-1]
portfolio['Average Weekly Drawdown'] = average_weekly_drawdown.append()
portfolio
'''
average_weekly_drawdowns = drawdowns.rolling(window=5).mean().iloc[-1]
portfolio['Average Weekly Drawdown'] = average_weekly_drawdown
portfolio



,Portfolio Weight (%),Annualized Volatility,Beta Against SPY,Beta Against IWM,Beta Against DIA,Average Weekly Drawdown
NVDA,0.07,0.634418,2.214848,1.839605,2.376997,42.095012
TSLA,0.07,0.557269,1.780724,1.539481,1.788735,313.493467
GME,0.07,0.746471,1.957305,1.972465,2.045533,2.398628
AMD,0.07,0.642289,2.067816,1.721387,2.218046,152.996285
MSFT,0.07,0.400705,1.259122,0.916869,1.393819,1338.645560
META,0.07,0.716671,1.702263,1.337453,1.773996,251.827006
WMT,0.07,0.246433,0.428176,0.265739,0.570775,316.223712


In [445]:
maximum_weekly_drawdown = drawdowns.rolling(window=5).max().iloc[-1]
portfolio['Maximum Weekly Drawdown'] = maximum_weekly_drawdown
portfolio

,Portfolio Weight (%),Annualized Volatility,Beta Against SPY,Beta Against IWM,Beta Against DIA,Average Weekly Drawdown,Maximum Weekly Drawdown
NVDA,0.07,0.634418,2.214848,1.839605,2.376997,42.095012,43.106507
TSLA,0.07,0.557269,1.780724,1.539481,1.788735,313.493467,329.974925
GME,0.07,0.746471,1.957305,1.972465,2.045533,2.398628,2.521421
AMD,0.07,0.642289,2.067816,1.721387,2.218046,152.996285,156.500007
MSFT,0.07,0.400705,1.259122,0.916869,1.393819,1338.645560,1353.504558
META,0.07,0.716671,1.702263,1.337453,1.773996,251.827006,255.505948
WMT,0.07,0.246433,0.428176,0.265739,0.570775,316.223712,319.981791


In [446]:
# Total Return = (final - initial) / (initial)

total_return = (data1.iloc[-1] - data1.iloc[252])/data1.iloc[252]
portfolio['Total Return'] = total_return
portfolio


,Portfolio Weight (%),Annualized Volatility,Beta Against SPY,Beta Against IWM,Beta Against DIA,Average Weekly Drawdown,Maximum Weekly Drawdown,Total Return
NVDA,0.07,0.634418,2.214848,1.839605,2.376997,42.095012,43.106507,43.763331
TSLA,0.07,0.557269,1.780724,1.539481,1.788735,313.493467,329.974925,19.997282
GME,0.07,0.746471,1.957305,1.972465,2.045533,2.398628,2.521421,1.945622
AMD,0.07,0.642289,2.067816,1.721387,2.218046,152.996285,156.500007,21.529941
MSFT,0.07,0.400705,1.259122,0.916869,1.393819,1338.645560,1353.504558,6.677201
META,0.07,0.716671,1.702263,1.337453,1.773996,251.827006,255.505948,1.410556
WMT,0.07,0.246433,0.428176,0.265739,0.570775,316.223712,319.981791,1.320966


In [447]:
'''
Annualized Total Return: [(1 + R_n)^(1/n) - 1]
Credit for Formula: https://corporatefinanceinstitute.com/resources/wealth-management/annualized-total-return/
'''

# 1 + R_n is a series until there are no more values hence why we use data.iloc functionality instead of 1 + R_1... There are many values
annualized_total_return = ((data1.iloc[-1] / data1.iloc[252])**(1/10) - 1) * 100
portfolio['Annualized Total Return (%)'] = annualized_total_return
portfolio

,Portfolio Weight (%),Annualized Volatility,Beta Against SPY,Beta Against IWM,Beta Against DIA,Average Weekly Drawdown,Maximum Weekly Drawdown,Total Return,Annualized Total Return (%)
NVDA,0.07,0.634418,2.214848,1.839605,2.376997,42.095012,43.106507,43.763331,46.248776
TSLA,0.07,0.557269,1.780724,1.539481,1.788735,313.493467,329.974925,19.997282,35.586455
GME,0.07,0.746471,1.957305,1.972465,2.045533,2.398628,2.521421,1.945622,11.408341
AMD,0.07,0.642289,2.067816,1.721387,2.218046,152.996285,156.500007,21.529941,36.545064
MSFT,0.07,0.400705,1.259122,0.916869,1.393819,1338.645560,1353.504558,6.677201,22.608419
META,0.07,0.716671,1.702263,1.337453,1.773996,251.827006,255.505948,1.410556,9.197255
WMT,0.07,0.246433,0.428176,0.265739,0.570775,316.223712,319.981791,1.320966,8.784466


Table showing constituent (stocks) risk analysis in the equal-weight portofolio analysis as of the current date

In [448]:
portfolio

,Portfolio Weight (%),Annualized Volatility,Beta Against SPY,Beta Against IWM,Beta Against DIA,Average Weekly Drawdown,Maximum Weekly Drawdown,Total Return,Annualized Total Return (%)
NVDA,0.07,0.634418,2.214848,1.839605,2.376997,42.095012,43.106507,43.763331,46.248776
TSLA,0.07,0.557269,1.780724,1.539481,1.788735,313.493467,329.974925,19.997282,35.586455
GME,0.07,0.746471,1.957305,1.972465,2.045533,2.398628,2.521421,1.945622,11.408341
AMD,0.07,0.642289,2.067816,1.721387,2.218046,152.996285,156.500007,21.529941,36.545064
MSFT,0.07,0.400705,1.259122,0.916869,1.393819,1338.645560,1353.504558,6.677201,22.608419
META,0.07,0.716671,1.702263,1.337453,1.773996,251.827006,255.505948,1.410556,9.197255
WMT,0.07,0.246433,0.428176,0.265739,0.570775,316.223712,319.981791,1.320966,8.784466


In [449]:
# Creating a table showing Portfolio Risk against the three ETFS
etf_portfolio = pd.DataFrame(index=etfs)
etf_portfolio

""
SPY
IWM
DIA


In [450]:
# Credit: https://github.com/CCNY-Analytics-and-Quant/PortfolioAnalysis-Rifat_Kaljang/blob/main/PortfolioAnalysis.ipynb

portfolioCorr = (data[tickers].pct_change() * portfolioWeight).sum(axis=1)
portfolioCorr = portfolioCorr.rename("Portfolio")
returns = pd.concat([data.pct_change(), portfolioCorr], axis = 1)

corr_matrix = returns[252:].corr(method="pearson")

etf_portfolio['Correlation against ETF'] = [corr_matrix['Portfolio'][etf] for etf in etfs]

etf_cov = returns[252:].cov()
etf_portfolio['Covariance against ETF'] = [etf_cov['Portfolio'][etf] for etf in etfs]
etf_portfolio



,Correlation against ETF,Covariance against ETF
SPY,0.706765,0.000077
IWM,0.668021,0.000091
DIA,0.625429,0.000069


In [451]:
'''
Tracking Error: Standard Deviation of (P - B) where P = Portfolio Returns and B = Benchmark Returns
Formula: Standard Deviation of (Stock Return - ETF return)
Measures how consistently close or wide an index ETF's performance is relative to its benchmark
Source: https://corporate.vanguard.com/content/corporatesite/us/en/corp/articles/tracking-error-often-overlooked-cost.html#:~:text=Tracking%20error%20is%20measured%20as,around%20performance%20adds%20uncertainty%20costs.
'''

etf_portfolio['Tracking Errors'] = [(returns['Portfolio'] - returns[etf]).std() for etf in etfs]
etf_portfolio


,Correlation against ETF,Covariance against ETF,Tracking Errors
SPY,0.706765,0.000077,0.007969
IWM,0.668021,0.000091,0.010187
DIA,0.625429,0.000069,0.008936


In [452]:
'''
Sharpe Ratio:  (Expected Asset Return - Risk Free Rate) / Standard Deviation
#Formula: Sharpe Ratio = (Rx - Rf) / StdDev(Rx)
Expected Asset Return = Portfolio Stock, Risk Free Rate = ETFs (benchmarks)
'''

etf_portfolio['Sharpe Ratio'] = [(returns['Portfolio'].mean() - returns[etf].mean())/(returns['Portfolio'].std()) for etf in etfs]
etf_portfolio


,Correlation against ETF,Covariance against ETF,Tracking Errors,Sharpe Ratio
SPY,0.706765,0.000077,0.007969,0.027550
IWM,0.668021,0.000091,0.010187,0.034955
DIA,0.625429,0.000069,0.008936,0.029568


In [453]:
# Annualized Volatlity
# Comparing asset portfolio to benchmark (ETF) portfolio
# portfolio volatility - etf volatility

'''
**REFERENCE CODE SO I DON'T HAVE TO SCROLL UP

portfolio['Annualized Volatility'] = data.pct_change()[-63:].std() * np.sqrt(252)
portfolio
'''

etf_portfolio['Annualized Volatility'] = [(returns['Portfolio'][252:].std())*(np.sqrt(252)) - (returns[etf][252:].std())*(np.sqrt(252)) for etf in etfs]
etf_portfolio

,Correlation against ETF,Covariance against ETF,Tracking Errors,Sharpe Ratio,Annualized Volatility
SPY,0.706765,0.000077,0.007969,0.027550,-0.024024
IWM,0.668021,0.000091,0.010187,0.034955,-0.068011
DIA,0.625429,0.000069,0.008936,0.029568,-0.024560


In [454]:
# Thanks ChatGPT for making it easy to input the values
# I could have used Matplotlib/Seaborn to make it easier to create a correlation matrix but I wanted to strictly use 3 libraries only for this project

asset_portfolio = {
    'NVDA': [0.07, 0.634418, 2.214848, 1.839605, 2.376997, 42.095012, 43.106507, 43.763331, 46.248776],
    'TSLA': [0.07, 0.557269, 1.780724, 1.539481, 1.788735, 313.493467, 329.974925, 19.997282, 35.586455],
    'GME': [0.07, 0.746471, 1.957305, 1.972465, 2.045533, 2.398628, 2.521421, 1.945622, 11.408341],
    'AMD': [0.07, 0.642289, 2.067816, 1.721387, 2.218046, 152.996285, 156.500007, 21.529941, 36.545064],
    'MSFT': [0.07, 0.400705, 1.259122, 0.916869, 1.393819, 1338.645560, 1353.504558, 6.677201, 22.608419],
    'META': [0.07, 0.716671, 1.702263, 1.337453, 1.773996, 251.827006, 255.505948, 1.410556, 9.197255],
    'WMT': [0.07, 0.246433, 0.428176, 0.265739, 0.570775, 316.223712, 319.981791, 1.320966, 8.784466]
}

benchmark_portfolio = {
    'SPY': [0.706765, 0.000077, 0.007969, 0.027550, -0.024024],
    'IWM': [0.668021, 0.000091, 0.010187, 0.034955, -0.068011],
    'DIA': [0.625429, 0.000069, 0.008936, 0.029568, -0.024560]
}

df_asset = pd.DataFrame(asset_portfolio)
df_benchmark = pd.DataFrame(benchmark_portfolio)

combined_portfolios = pd.concat([df_asset, df_benchmark],axis=1)
coMatrix = combined_portfolios.corr()

The Correlation Matrix between the equal-weighted portfolios

In [455]:
coMatrix

,NVDA,TSLA,GME,AMD,MSFT,META,WMT,SPY,IWM,DIA
NVDA,1.000000,0.631896,0.550152,0.705459,0.574489,0.579795,0.578284,-0.749520,-0.765624,-0.749695
TSLA,0.631896,1.000000,0.020737,0.994531,0.996936,0.997575,0.997376,-0.766958,-0.777662,-0.766310
GME,0.550152,0.020737,1.000000,0.109415,-0.043705,-0.031157,-0.035940,-0.797331,-0.804478,-0.795935
AMD,0.705459,0.994531,0.109415,1.000000,0.984161,0.985705,0.985227,-0.762294,-0.778094,-0.762492
MSFT,0.574489,0.996936,-0.043705,0.984161,1.000000,0.999913,0.999968,-0.744853,-0.766588,-0.746280
META,0.579795,0.997575,-0.031157,0.985705,0.999913,1.000000,0.999982,-0.821757,-0.836291,-0.822337
WMT,0.578284,0.997376,-0.035940,0.985227,0.999968,0.999982,1.000000,-0.751257,-0.787710,-0.756054
SPY,-0.749520,-0.766958,-0.797331,-0.762294,-0.744853,-0.821757,-0.751257,1.000000,0.997537,0.999938
IWM,-0.765624,-0.777662,-0.804478,-0.778094,-0.766588,-0.836291,-0.787710,0.997537,1.000000,0.998158
DIA,-0.749695,-0.766310,-0.795935,-0.762492,-0.746280,-0.822337,-0.756054,0.999938,0.998158,1.000000
